In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("autodl-tmp/Mistral-7B-Instruct-v0.1", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("autodl-tmp/Mistral-7B-Instruct-v0.1", trust_remote_code=True)

len(tokenizer("hello")["input_ids"])

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


3

In [2]:
template_ = [
  {
    "instruction": "user instruction",
    "input": "user input",
    "output": [
      "chosen answer",
      "rejected answer"
    ]
  }
]

In [2]:
# zero-shot
instruction = ( 
    "You are a web navigation intelligence who interacts with webpage environments to achieve human user intent.\n"
    "You always generate the next ACTION based on the user's INTENT, current webpage HTML and ACTION_HISTORY sequence which recording the actions that have been performed.\n\n"
    "Given HTML and INTENT and ACTION_HISTORY, you should\n"
    "(1) Rely on your HTML code comprehension to analyze and understand what elements are on the current page.\n"
    "(2) Depend on your reasoning skills to parse the user's INTENT and infer the next action that should be taken in conjunction with the historical trajectory ACTION_HISTORY.\n"
    "(3) Select an element carefully from HTML code to interact with, thus bringing the goal closer to completion.\n\n"
    "Your output format should be strictly as follows\n"
    "Operation: ... (should be CLICK or TYPE)\n"
    "Value: ... (optional textual value for the operation TYPE)\n"
    "ID: ... (unique id number for the element to click or type into)\n\n"
    # "Thought: ... (A paragraph explaining why you chose this element to interact with, no more than 50 words)"
    "Now, begin!"
)

print(instruction)

You are a web navigation intelligence who interacts with webpage environments to achieve human user intent.
You always generate the next ACTION based on the user's INTENT, current webpage HTML and ACTION_HISTORY sequence which recording the actions that have been performed.

Given HTML and INTENT and ACTION_HISTORY, you should
(1) Rely on your HTML code comprehension to analyze and understand what elements are on the current page.
(2) Depend on your reasoning skills to parse the user's INTENT and infer the next action that should be taken in conjunction with the historical trajectory ACTION_HISTORY.
(3) Select an element carefully from HTML code to interact with, thus bringing the goal closer to completion.

Your output format should be strictly as follows
Operation: ... (should be CLICK or TYPE)
Value: ... (optional textual value for the operation TYPE)
ID: ... (unique id number for the element to click or type into)

Now, begin!


In [4]:
int(len(instruction) / 4)

236

In [3]:
user_input_template = "INTENT:\n{intent}\n\nHTML:\n{html}\n\nACTION_HISTORY:\n{action_history}"
output_template = "Operation: {op}\nValue: {value}\nID: {id}"

In [6]:
# unfold the trajectory 
# change the backend_node_id to id for token reduction...
# CL 1:3 percentage
# drop the too long html data sample

train_dataset = []

In [8]:
import json
import datasets

import random
from tqdm import tqdm

for ID in range(11):
    with open(f"autodl-tmp/train_dataset/train_{ID}.json") as file:
        data = json.load(file)

    pos_candidate_na = 0
    total_dataset_num = 0
    
    for dat in tqdm(data):
        intent = dat["confirmed_task"] # + f"(domain {dat['subdomain']})"
        action_history_all = dat["action_reprs"]
    
        for index, d in enumerate(dat["actions"]):
            cleaned_html = d["cleaned_html"].replace("backend_node_id", "id")
            token_num = len(tokenizer(cleaned_html)["input_ids"])
    
            if token_num > 120000:
                print(f"oversize intent: {intent}")
            else:
                html = cleaned_html
                action_history = action_history_all[:index]
    
                op = d["operation"]["op"]
                value = d["operation"]["value"]
                if len(d["pos_candidates"]) > 0:
                    chosen_answer_ = output_template.format(op=op, value=value, id=d["pos_candidates"][0]["backend_node_id"])
                else:
                    pos_candidate_na += 1
                    continue
                
                random.shuffle(d["neg_candidates"])
                rand_neg_candidates = d["neg_candidates"][:3]
                
                for c in rand_neg_candidates: # 1:3 proportion
                    if op != "CLICK" and random.uniform(0, 1) < 0.33: # 1/3 for type/select -> click
                        rejected_answer_ = output_template.format(op="CLICK", value="", id=c["backend_node_id"])
                    else:
                        rejected_answer_ = output_template.format(op=op, value=value, id=c["backend_node_id"])
                        
                    instruction_ = instruction
                    input_ = user_input_template.format(intent=intent, html=html, action_history=action_history)
                    output_ = [chosen_answer_, rejected_answer_]
                    
                    total_dataset_num += 1
                    train_dataset.append({
                        "instruction": instruction_,
                        "input": input_,
                        "output": output_
                    })

    print(ID, pos_candidate_na, total_dataset_num)

 18%|█▊        | 18/100 [00:48<02:47,  2.04s/it]

oversize intent: See today's deals on video games and find the best deal on the lowest-priced mouse.


 19%|█▉        | 19/100 [00:53<03:55,  2.91s/it]

oversize intent: Find Metformin 1000mg tablet price and details.
oversize intent: Find Metformin 1000mg tablet price and details.
oversize intent: Find Metformin 1000mg tablet price and details.


 28%|██▊       | 28/100 [01:08<01:38,  1.37s/it]

...

oversize intent: Find a Chevrolet with lowest mileage in zip 08817 with shipping charges of 99 dollars or less, with a price between 20k to 30k.


100%|██████████| 100/100 [03:31<00:00,  2.11s/it]


9 28 2181


100%|██████████| 9/9 [00:09<00:00,  1.02s/it]

10 3 138


In [9]:
with open("autodl-tmp/mind2web_dpo_train.json", "w") as file:
    json.dump(train_dataset, file, indent=4)

In [10]:
test_task_dataset = []

In [11]:
import json
import datasets

import random
from tqdm import tqdm

for ID in range(3):
    with open(f"autodl-tmp/test_dataset/test_task/test_task_{ID}.json") as file:
        data = json.load(file)

    pos_candidate_na = 0
    total_dataset_num = 0
    
    for dat in tqdm(data):
        intent = dat["confirmed_task"] # + f"(domain {dat['subdomain']})"
        action_history_all = dat["action_reprs"]
    
        for index, d in enumerate(dat["actions"]):
            cleaned_html = d["cleaned_html"].replace("backend_node_id", "id")
            token_num = len(tokenizer(cleaned_html)["input_ids"])
    
            if token_num > 120000:
                print(f"oversize intent: {intent}")
            else:
                html = cleaned_html
                action_history = action_history_all[:index]
    
                op = d["operation"]["op"]
                value = d["operation"]["value"]
                if len(d["pos_candidates"]) > 0:
                    chosen_answer_ = output_template.format(op=op, value=value, id=d["pos_candidates"][0]["backend_node_id"])
                else:
                    pos_candidate_na += 1
                    continue
                
                # random.shuffle(d["neg_candidates"])
                # rand_neg_candidates = d["neg_candidates"][:3]
                
                # for c in rand_neg_candidates: # 1:3 proportion
                #     if op != "CLICK" and random.uniform(0, 1) < 0.33: # 1/3 for type/select -> click
                #         rejected_answer_ = output_template.format(op="CLICK", value="", id=c["backend_node_id"])
                #     else:
                #         rejected_answer_ = output_template.format(op=op, value=value, id=c["backend_node_id"])
                        
                instruction_ = instruction
                input_ = user_input_template.format(intent=intent, html=html, action_history=action_history)
                output_ = chosen_answer_
                
                total_dataset_num += 1
                test_task_dataset.append({
                    "instruction": instruction_,
                    "input": input_,
                    "output": output_
                })

    print(ID, pos_candidate_na, total_dataset_num)

  5%|▌         | 5/100 [00:11<03:43,  2.35s/it]

oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile
oversize intent: search gas pi

  6%|▌         | 6/100 [00:28<11:10,  7.13s/it]

oversize intent: search gas pickup truck in Fremont with 2010 and 2017 with less than 80000 mile


 12%|█▏        | 12/100 [00:43<04:26,  3.02s/it]

...

oversize intent: Show me products from Calvin Klein brand menswear list.


 98%|█████████▊| 51/52 [01:38<00:01,  1.31s/it]

oversize intent: Add the cheapest SSD to my cart
oversize intent: Add the cheapest SSD to my cart
oversize intent: Add the cheapest SSD to my cart


100%|██████████| 52/52 [01:46<00:00,  2.06s/it]

oversize intent: Add the cheapest SSD to my cart
2 16 435


In [12]:
with open("autodl-tmp/mind2web_dpo_test_task.json", "w") as file:
    json.dump(test_task_dataset, file, indent=4)